<a href="https://colab.research.google.com/github/the-ultimate-krol/studia-repo/blob/main/Wprowadzenie_do_transformers%C3%B3w_maskowanie_laboratorium_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wprowadzenie do architektury transformatorów

## W poprzednich odcinkach

Obecnie najpopularniejsze modele przetwarzania języka naturalnego wykorzystują architekturę Transformers. Istnieje kilka bibliotek implementujących tę architekturę. Jednak w kontekście NLP najczęściej używane są transformatory Huggingface.

Oprócz samego kodu źródłowego, biblioteka ta zawiera szereg innych elementów. Do najważniejszych z nich należą

[models](https://huggingface.co/models) -  ogromna i stale rosnąca liczba gotowych modeli, które możemy wykorzystać do rozwiązywania wielu problemów w NLP (ale także w rozpoznawaniu mowy czy przetwarzaniu obrazów),
[datasets](https://huggingface.co/datasets) - bardzo duży katalog przydatnych zbiorów danych, które możemy łatwo wykorzystać do trenowania własnych modeli NLP (i nie tylko).

## Przygotowanie środowiska - jak zacząć z Google Colab

Trenowanie modeli NLP wymaga dostępu do akceleratorów sprzętowych przyspieszających uczenie się sieci neuronowych. Jeśli nasz komputer nie jest wyposażony w GPU, możemy skorzystać ze środowiska Google Colab.

W środowisku tym możemy wybrać akcelerator spośród GPU i TPU. Sprawdźmy, czy mamy dostęp do środowiska wyposażonego w akcelerator NVidia, wykonując poniższe polecenie:

In [ ]:
!git clone https://github.com/jasonmayes/headless-chrome-nvidia-t4-gpu-support.git
!cd headless-chrome-nvidia-t4-gpu-support && chmod +x scriptyMcScriptFace.sh && ./scriptyMcScriptFace.sh

Następnie zainstalujemy wszystkie niezbędne biblioteki. Oprócz samej biblioteki `transformers`, zainstalujemy również `datasets` - bibliotekę zarządzającą zestawami danych, bibliotekę definiującą wiele metryk używanych w algorytmach AI `evaluate` oraz dodatkowe narzędzia, takie jak `sacremoses` i `sentencepiece`.

In [ ]:
!pip install transformers sacremoses datasets evaluate sentencepiece

Po zainstalowaniu niezbędnych bibliotek możemy korzystać ze wszystkich modeli i zbiorów danych zarejestrowanych w katalogu.

Typowym sposobem wykorzystania dostępnych modeli jest:

- korzystanie z gotowego modelu realizującego konkretne zadanie, np. [analiza wydźwięku](https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis) - model tego typu nie wymaga trenowania, wystarczy go uruchomić, aby uzyskać wynik klasyfikacji (można to zobaczyć w demo pod wskazanym linkiem),
- wykorzystanie modelu bazowego, który został wytrenowany do konkretnego zadania; przykładem takiego modelu jest [HerBERT base](https://huggingface.co/allegro/herbert-base-cased), który został nauczony jako model językowy wykorzystujący maskowanie. Aby użyć go do konkretnego zadania, musimy wybrać dla niego "głowicę klasyfikacyjną" i przetrenować go na naszym własnym zbiorze danych.

Modele tego rodzaju są różne i mogą być ładowane przy użyciu wspólnego interfejsu, ale najlepiej jest użyć jednej z wyspecjalizowanych klas, dostosowanej do danego zadania (https://boinc-ai.gitbook.io/transformers/api/main-classes/auto-classes/natural-language-processing/automodelformaskedlm). Zaczniemy od załadowania modelu bazowego BERT - jednego z najpopularniejszych modeli dla języka angielskiego. Użyjemy go do odgadnięcia brakujących słów w tekście. W tym celu użyjemy wywołania `AutoModelForMaskedLM`, co można zobaczyć po wywołaniu kodu:

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model = AutoModelForMaskedLM.from_pretrained("bert-base-cased")

## Łączenie z Google Drive
Ostatnim elementem przygotowań, który jest opcjonalny, jest dołączenie własnego Dysku Google do środowiska Colab. Umożliwia to zapisywanie wytrenowanych modeli podczas procesu szkolenia na "zewnętrznym" dysku. Jeśli Google Colab doprowadzi do przerwania procesu szkolenia, pliki, które zostały pomyślnie zapisane podczas szkolenia, nie zostaną utracone. Możliwe będzie wznowienie treningu już na częściowo wytrenowanym modelu.

Aby to zrobić, montujemy Dysk Google w Colab. Wymaga to autoryzacji narzędzia Colab na Dysku Google.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Po zamontowaniu dysku mamy dostęp do całej zawartości Dysku Google. Wskazując, gdzie zapisać dane podczas treningu, należy wskazać ścieżkę zaczynającą się od `/content/gdrive` i wskazać jakiś podkatalog w naszej przestrzeni dyskowej. Pełną ścieżką może być `/content/gdrive/MyDrive/output`. Dobrym pomysłem jest sprawdzenie, czy dane są zapisywane na dysku przed uruchomieniem treningu.

## Tokenizacja tekstu
Samo załadowanie modelu nie wystarczy jednak, by zacząć z niego korzystać. Musimy dysponować mechanizmem konwersji tekstu (ciągu znaków), na sekwencję tokenów, należących do określonego słownika. Podczas uczenia modelu słownik ten jest określany (wybierany algorytmicznie) przed właściwym uczeniem sieci neuronowej. Chociaż możliwe jest jego późniejsze rozszerzenie (trening na danych treningowych; pozwala również uzyskać reprezentację brakujących tokenów), zwykle używany jest słownik w formie zdefiniowanej przed treningiem sieci neuronowej. Dlatego ważne jest, aby określić prawidłowy słownik dla tokenizera wykonującego dzielenie tekstu.

Biblioteka posiada klasę `AutoTokenizer`, która akceptuje nazwę modelu, co pozwala na automatyczne załadowanie słownika odpowiadającego wybranemu modelowi sieci neuronowej. Należy jednak pamiętać, że w przypadku korzystania z dwóch modeli, każdy z nich najprawdopodobniej będzie miał inny słownik, a zatem muszą one mieć własne instancje klasy `Tokenizer` (https://huggingface.co/docs/transformers/main_classes/tokenizer).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Tokenizer używa słownika o stałym rozmiarze, co powoduje, że nie wszystkie słowa występujące w tekście zostaną uwzględnione. A jeśli użyjemy tokenizera do podzielenia tekstu w języku innym niż ten, dla którego został stworzony, taki tekst zostanie podzielony na większą liczbę tokenów. (https://huggingface.co/docs/transformers/main_classes/tokenizer)

In [ ]:
sentence1 = tokenizer.encode(
    "The quick brown fox jumps over the lazy dog.", return_tensors="pt"
)
print(sentence1)
print(sentence1.shape)

sentence2 = tokenizer.encode("Zażółć gęślą jaźń.", return_tensors="pt")
print(sentence2)
print(sentence2.shape)

Używając tokenizera dla języka angielskiego do podzielenia zdania w dowolnym innym języku, widzimy, że otrzymujemy znacznie większą liczbę tokenów. Aby zobaczyć, jak tokenizator podzielił tekst, możemy użyć wywołania `covert_ids_to_tokens`:

In [ ]:
print("|".join(tokenizer.convert_ids_to_tokens(list(sentence1[0]))))
print("|".join(tokenizer.convert_ids_to_tokens(list(sentence2[0]))))

Widzimy, że w przypadku języka angielskiego wszystkie słowa w zdaniu zostały przekonwertowane na pojedyncze tokeny. W przypadku zdania w dowolnym innym języku zawierającym szereg znaków diakrytycznych, sytuacja wygląda zupełnie inaczej - każdy znak został wyodrębniony w osobny pod-token. Fakt, że mamy do czynienia z pod-tokenami jest sygnalizowany przez dwa krzyżyki poprzedzające dany pod-token. Wskazują one, że ten pod-token musi zostać sklejony z poprzedzającym go tokenem w celu uzyskania poprawnego ciągu znaków).

## Ćwiczenie

Użyj tokenizera dla `xlm-roberta-large` do tokenizacji tych samych zdań. Jakie wnioski można wyciągnąć, patrząc na sposób tokenizacji przy użyciu różnych słowników?

In [ ]:
# your code

W wyniku tokenizacji, oprócz słów/tokenów obecnych w oryginalnym tekście, w wynikach tokenizacji pojawiają się dodatkowe znaczniki [CLS] i [SEP] (lub inne znaczniki - w zależności od używanego słownika). Mają one specjalne znaczenie i mogą być wykorzystywane do wykonywania określonych funkcji związanych z analizą tekstu. Na przykład reprezentacja tokenu [CLS] jest używana w zadaniach klasyfikacji zdań. Z kolei token [SEP] służy do rozróżniania zdań w zadaniach wymagających dwóch zdań jako danych wejściowych (np. określania, jak bardzo zdania są do siebie podobne).

##Modelowanie języka

Modele wstępnie przetworzone w reżimie samonadzorowanego uczenia się (SSL) nie mają specjalnych możliwości rozwiązywania określonych zadań przetwarzania języka naturalnego, takich jak odpowiadanie na pytania lub klasyfikowanie tekstu (z wyjątkiem bardzo dużych modeli, takich jak na przykład GPT-3). Mogą być jednak wykorzystywane do określania prawdopodobieństwa występowania słów w tekście, a tym samym do testowania, jak dużą wiedzę posiada dany model w zakresie znajomości języka lub ogólnej wiedzy o świecie.

Aby sprawdzić, jak model radzi sobie w tych zadaniach, możemy przeprowadzić wnioskowanie na danych wejściowych, w których niektóre słowa zostaną zastąpione specjalnymi symbolami maskującymi używanymi podczas wstępnego szkolenia modelu.


Należy pamiętać, że różne modele mogą używać różnych sekwencji specjalnych podczas szkolenia wstępnego. Na przykład Bert używa sekwencji [MASK]. Możemy sprawdzić wygląd tokena maski lub jego identyfikator w [pliku konfiguracyjnym tokenizera](https://huggingface.co/bert-base-cased/raw/main/tokenizer.json) dystrybuowanym wraz z modelem.

W pierwszym kroku spróbujemy uzupełnić brakujące słowo w zdaniu w języku angielskim.

In [ ]:
sentence_en = tokenizer.encode(
    "The quick brown [MASK] jumps over the lazy dog.", return_tensors="pt"
)
print("|".join(tokenizer.convert_ids_to_tokens(list(sentence_en[0]))))
target = model(sentence_en)
print(target.logits[0][4])

Ponieważ zdanie jest uzupełnione tagiem `[CLS]` po stocenizacji, zamaskowane słowo znajduje się na pozycji 4. Wywołanie `call target.logits[0][4]` pokazuje tensor z rozkładem prawdopodobieństwa poszczególnych słów, który został określony na podstawie parametrów modelu. Możemy wybrać słowa o najwyższym prawdopodobieństwie za pomocą wywołania `torch.topk`:

In [ ]:
import torch

top = torch.topk(target.logits[0][4], 5)
top

Otrzymaliśmy dwa wektory - `wartości` zawierające składowe wektora wyjściowego sieci neuronowej (nieznormalizowane) oraz `indeksy` zawierające indeksy tych składowych. Na tej podstawie możemy wyświetlić wyrażenie, które według modelu jest najbardziej prawdopodobnym uzupełnieniem zamaskowanego wyrażenia:

In [ ]:
words = tokenizer.convert_ids_to_tokens(top.indices)

import matplotlib.pyplot as plt
plt.bar(words, top.values.detach().numpy())

Zgodnie z oczekiwaniami, najbardziej prawdopodobnym zamiennikiem brakującego słowa jest pies. Drugie słowo ##ie może być nieco zaskakujące, ale po dodaniu do istniejącego tekstu otrzymujemy zdanie "The quick brownie jumps over the lazy dog", które również wydaje się sensowne (choć nieco zaskakujące).

## Ćwiczenie

Używając `xlm-roberta-model`, zaproponuj zdania z jednym brakującym słowem, weryfikując zdolność tego modelu do:

uwzględnienia znaczenia w kontekście semantycznym,
uwzględnienia relacji długodystansowych w tekście,
reprezentowania wiedzy o świecie.
Dla każdego problemu wymyśl 3 zdania testowe i wyświetl przewidywania dla 5 najbardziej prawdopodobnych słów.

Postaraj się wymyślić przykłady z zamaskowanym elementem w różnych pozycjach w zdaniu.

Możesz użyć kodu z funkcji `plot_words`, aby wyświetlić wyniki. Sprawdź również, jaki token maskujący jest używany w tym modelu i pamiętaj o załadowaniu `xlm-roberta-model`.

In [ ]:
def plot_words(sentence, word_model, word_tokenizer, mask="[MASK]"):
    sentence = word_tokenizer.encode(sentence, return_tensors="pt")
    tokens = word_tokenizer.convert_ids_to_tokens(list(sentence[0]))
    print("|".join(tokens))
    target = word_model(sentence)
    top = torch.topk(target.logits[0][tokens.index(mask)], 5)
    words = word_tokenizer.convert_ids_to_tokens(top.indices)
    plt.xticks(rotation=45)
    plt.bar(words, top.values.detach().numpy())
    plt.show()


# your code